In [7]:
from src.backend.retrieval.file_reader import read_csv
from src.backend.retrieval.pipeline import TwoStagePipeline, preprocess_text_column

In [8]:
documents = read_csv('data/processed/all_data_cleaned.csv', text_column='cleaned_text')
type(documents)
documents

0         Approximately 100km long firebreaks have been ...
1                                          God bless you...
2         RT : Cracked wine casks, damaged historical  b...
3         I'm really just excited for new undies and pin...
4         Rescue effort expands in India, Pakistan as fl...
                                ...                        
212761        Tips to avoid flood damaged cars after Harvey
212762    you don't have to say that twice... Over here ...
212763    RT : We stand with  in the aftermath of the ea...
212764             Quake: M 5.1, southern East Pacific Rise
212765    Of to do skydive with   landing on a beach can...
Name: cleaned_text, Length: 212766, dtype: object

In [9]:
documents = preprocess_text_column(documents)

Token indices sequence length is longer than the specified maximum sequence length for this model (932 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 